In [1]:
# Importing required libraries

import time

from bs4 import BeautifulSoup
from selenium import webdriver

from pymongo import MongoClient

In [2]:
# Creating a connection to MongoDB
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador']

In [3]:
# Base URL of the site to be analyzed
SITE_URL = 'https://www.elespectador.com'

In [4]:
# Firefox web driver path
# Download the driver for you S.O. here: https://github.com/mozilla/geckodriver/releases
DRIVER_PATH = './geckodriver.exe'

In [5]:
# Creating a new firefox window
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

C:\Users\JCristancho\AppData\Local\Temp\ipykernel_13332\1910320475.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path = DRIVER_PATH)


In [6]:
def make_request(browser, relative_path):
    # Making the request and rendering the browser
    browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis
    return BeautifulSoup(browser.page_source, 'html.parser')

In [7]:
# Getting HTML content for news listing page
soup = make_request(browser, '/archivo/politica/1')

In [8]:
# Finding the section where news are contained 
layout = soup.find(class_ = 'Layout-flexAds')

In [9]:
# Getting blocks from layout
blocks = layout.find('section').find_all(recursive = False)
print(len(blocks)) # 3 blocks founded, 2 for news and 1 for pagintion

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [ ]:
# Finding and concatenating news cards
cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
len(cards)

In [ ]:
# Building a list with title and relative path of the news founded

news = []

for card in cards:
    news.append({
        'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
        'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
    })

In [ ]:
news

In [ ]:
for n in news:
    # Getting HTML content for news page
    soup = make_request(browser, n['relative_path'])
   
    
    # conditional that determines if the article contains an author or not
    if soup.find(class_ = 'ACredit-Author').find('a') is not None:
        
        # Extracting news metadata
        n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
        n['author'] = soup.find(class_ = 'ACredit-Author').find('a').get_text()
        n['summary'] =soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
        
    else:
        n['author'] = 'no author'
    
    soup1 = soup
    # remove <a> tag from html
    [s.extract() for s in soup1("a")]
    # Extracting and concatenating news full text
    paragraphs = soup1.find_all(class_ = 'font--secondary')
    
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])



In [ ]:
news

In [ ]:
# Storing extracted information for further analysis
collection.insert_many(news)